In [1]:
import pandas as pd

In [2]:
la_ho_data = pd.read_csv("/mnt/c/Users/ethan/la_ho_data.csv")

In [3]:
la_ho_data.head()

,Last Name,First Name,Middle Initial,DOC Number,Gender,Race,Current Age,Birth Date,Offense Statute,Modifier 1,Modifier 2,Drug Activity,Offense Date,Sentence Date,Imposed Length,Overall Sentence Length,Parish Literal
0,GRAY,JOSEPH,NaN,72486,M,B,81.90,19400110,14:64,H,NaN,NaN,19880408.0,19890105.0,LIFE,LIFE,IBERVILLE
1,GRAY,JOSEPH,NaN,72486,M,B,81.90,19400110,14:64,NaN,NaN,NaN,19880229.0,19930712.0,0150000,LIFE,IBERVILLE
2,MARTIN,WENDELL,A,72879,M,B,68.94,19521225,14:31,NaN,NaN,NaN,19711101.0,19880811.0,0080000,LIFE,ORLEANS
3,MARTIN,WENDELL,A,72879,M,B,68.94,19521225,14:62,H,NaN,NaN,19780301.0,19880811.0,0150000,LIFE,ORLEANS
4,MARTIN,WENDELL,A,72879,M,B,68.94,19521225,14:30.1,NaN,NaN,NaN,19850424.0,19860210.0,LIFE,LIFE,ORLEANS


## Filtering the Data
We're interested in the charges that led to the habitual offender sentence, so we first have to filter the rows to only include those charges.

In [4]:
only_hos = la_ho_data.loc[
    (la_ho_data["Modifier 1"] == "H")
    | (la_ho_data["Modifier 2"] == "H")
]
only_hos

,Last Name,First Name,Middle Initial,DOC Number,Gender,Race,Current Age,Birth Date,Offense Statute,Modifier 1,Modifier 2,Drug Activity,Offense Date,Sentence Date,Imposed Length,Overall Sentence Length,Parish Literal
0,GRAY,JOSEPH,NaN,72486,M,B,81.90,19400110,14:64,H,NaN,NaN,19880408.0,19890105.0,LIFE,LIFE,IBERVILLE
3,MARTIN,WENDELL,A,72879,M,B,68.94,19521225,14:62,H,NaN,NaN,19780301.0,19880811.0,0150000,LIFE,ORLEANS
5,FINLEY,DOLPH,NaN,75033,M,B,72.38,19490719,14:64.1,H,NaN,NaN,19860609.0,20130118.0,0290629,0290629,RAPIDES
7,GORDON,DANIEL,NaN,78855,M,B,66.73,19550312,14:65,H,NaN,NaN,19961017.0,20110401.0,0140000,LIFE,ORLEANS
12,NICHOLAS,MICHAEL,NaN,81555,M,B,70.04,19511119,14:67,H,NaN,NaN,19960320.0,19970523.0,LIFE,LIFE,ORLEANS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,BARKER,SAMUEL,J,718022,M,W,65.29,19560818,14:62.2,A,H,NaN,20150614.0,20170720.0,0200000,LIFE,ORLEANS
2592,BARKER,SAMUEL,J,718022,M,W,65.29,19560818,14:67.B(1),H,NaN,NaN,20150613.0,20170720.0,0200000,LIFE,ORLEANS
2594,DEGREGORY,CRISTIAN,J,723089,M,W,40.81,19810209,14:93.2.3,H,NaN,NaN,20150526.0,20180126.0,0500000,0500000,ORLEANS
2596,PERKINS,WADE,H,725743,M,W,36.27,19850828,14:81.1,H,2,NaN,20160428.0,20170919.0,0260400,0260400,JEFFERSON


## Reformatting the Data
Since our goal here is to calculate the number of people serving sentences longer than 20 years for each offense/category of offenses, the first thing we want to do is create columsn for each offense category so that we don't have to keep track of each offense code: group the data by DOC number:

#### First-Degree Murder
People charged with first-degree murder have `14:30` in the `Offense Statute`. Note that `14:30.1` should _not_ be included, as this indicates second-degree murder. Because we didn't do much data cleaning, we need to double-check that there aren't any weirdly formatted offense codes that would be missed by a straightforward search:

In [5]:
only_hos.loc[
    only_hos["Offense Statute"].str.contains("14")
    & only_hos["Offense Statute"].str.contains("30"),
    "Offense Statute"
].value_counts()

14:30.1     41
14:30       24
14:130.1     9
Name: Offense Statute, dtype: int64

The check looks good, so a simple search should be fine.

In [6]:
first_degree_murder = (only_hos["Offense Statute"] == "14:30")

In [7]:
first_degree_murder.sum()

24

#### Second Degree Murder
We calculate this much the same way, just replacing `14:30` with `14:30.1`.

In [8]:
second_degree_murder = (only_hos["Offense Statute"] == "14:30.1")

In [9]:
second_degree_murder.sum()

41

### Rape
There are three statutes that correspond to rape: `14.42` (First degree rape), `14.42.1` (Second degree rape), and `14.43` (Third degree rape). Before doing a search for these codes, we perform a similar check to make sure there aren't any weirdly formated codes.

In [10]:
only_hos.loc[
    only_hos["Offense Statute"].str.contains("14")
    & only_hos["Offense Statute"].str.contains("42|43"),
    "Offense Statute"
].value_counts()

14:42.1    31
14:42      17
14:43.1    11
14:43       4
14:43.3     2
14:43.4     1
14:43.2     1
Name: Offense Statute, dtype: int64

Also looks good, but note that `14:43.X` should _not_ be included, as these refer to less serious offenses, such as misdemeanor sexual battery.

In [11]:
rape = (
    only_hos["Offense Statute"].str.match("14:42|14:43$")  # The $ excludes all of the 14.43.X codes
)

In [12]:
rape.sum()

52

#### Drug offenses
Drug offenses are scattered across a bunch of different offense codes. Fortunately, we have the `Drug Activity` column, so we don't need to rely on the offense codes.

In [13]:
only_hos["Drug Activity"].value_counts()

POSS    70
PWID    39
DIST    35
CWID     1
Name: Drug Activity, dtype: int64

Thus, we can ID all drug offenses based on whether or not the `Drug Activity` column is empty.

In [14]:
drugs = ~only_hos["Drug Activity"].isna()

In [15]:
drugs.sum()

145

#### Possession, PWID, and distribution

Similarly, we can rely on the `Drug Activity` column again here:

In [16]:
poss = only_hos["Drug Activity"] == "POSS"

In [17]:
poss.sum()

70

In [18]:
pwid = only_hos["Drug Activity"] == "PWID"

In [19]:
pwid.sum()

39

In [20]:
dist = only_hos["Drug Activity"] == "DIST"

In [21]:
dist.sum()

35

### Combining results
Now we can add these new columns to a new DataFrame:

In [22]:
revised_la_ho_data = only_hos.assign(
    first_degree_murder=first_degree_murder,
    second_degree_murder=second_degree_murder,
    rape=rape,
    drugs=drugs,
    poss=poss,
    pwid=pwid,
    dist=dist,
)
revised_la_ho_data.head()

,Last Name,First Name,Middle Initial,DOC Number,Gender,Race,Current Age,Birth Date,Offense Statute,Modifier 1,...,Imposed Length,Overall Sentence Length,Parish Literal,first_degree_murder,second_degree_murder,rape,drugs,poss,pwid,dist
0,GRAY,JOSEPH,NaN,72486,M,B,81.90,19400110,14:64,H,...,LIFE,LIFE,IBERVILLE,False,False,False,False,False,False,False
3,MARTIN,WENDELL,A,72879,M,B,68.94,19521225,14:62,H,...,0150000,LIFE,ORLEANS,False,False,False,False,False,False,False
5,FINLEY,DOLPH,NaN,75033,M,B,72.38,19490719,14:64.1,H,...,0290629,0290629,RAPIDES,False,False,False,False,False,False,False
7,GORDON,DANIEL,NaN,78855,M,B,66.73,19550312,14:65,H,...,0140000,LIFE,ORLEANS,False,False,False,False,False,False,False
12,NICHOLAS,MICHAEL,NaN,81555,M,B,70.04,19511119,14:67,H,...,LIFE,LIFE,ORLEANS,False,False,False,False,False,False,False


### Identifying Unique Persons
Our next challenge is counting the number of unique persons charged with each offense, as well the total number of unique people in the data set. We do this by grouping the data by `DOC Number`.

In [23]:
by_doc_no = revised_la_ho_data.groupby("DOC Number")

This lets us count the number of unique persons in the dataset:

In [24]:
len(by_doc_no.groups)

884

#### First Degree Murder
Now, we can count the number of people who have been charged with first degree murder:

In [25]:
by_doc_no["first_degree_murder"].any().sum()

23

#### Second Degree Murder
And again:

In [26]:
by_doc_no["second_degree_murder"].any().sum()

41

#### Rape
Same deal here:

In [27]:
by_doc_no["rape"].any().sum()

51

#### Drugs

In [28]:
by_doc_no["drugs"].any().sum()

126

#### Possession

In [29]:
by_doc_no["poss"].any().sum()

64

#### PWID

In [30]:
by_doc_no["pwid"].any().sum()

34

#### Distribution

In [31]:
by_doc_no["dist"].any().sum()

34

## One Big Complication
There's one big complication with using these numbers as is: The language in the draft implies that we're talking about the _most serious_ offense that each person is charged with, but it's possible, for instance, that someone charged with drug possession was also charged with murder, in which case it's misleading to say that they're in prison for drug possession. To rectify this, we identify the most serious charge for each person, based on sentence length.

### Calculating sentence length

This is a bit tricky, because the data uses a somewhat odd format for sentence lengths: "Sentence Lengths are read as yyy/mm/dd. 0290629 is 029 years, 6 months, and 29 days."
Thus, our first task is to create a more usable sentence-length column.

In [32]:
def parse_sentence_length(sentence_str):
    sentence_str = sentence_str.upper()  # some are lower case
    if (sentence_str == "DEATH"):
        return 999999999999999  # Just return a big number so that it's higher than any non-death sentence
    if (sentence_str == "LIFE"):
        return 100000000000000  # Just return a big number so that it's higher than any non-life sentence
    if not sentence_str:
        return pd.NA  # In case it's blank, we shouldn't try to parse
    if len(sentence_str) < 7:
        sentence_str = "0" + sentence_str  # Make sure the leading zeros didn't get cut off
    years = int(sentence_str[:3])
    months = int(sentence_str[3:5])
    days = int(sentence_str[5:])
    return years + months / 12 + days / 365

In [33]:
sentence_length = pd.to_numeric(
    revised_la_ho_data["Imposed Length"].fillna("").map(parse_sentence_length)
)

In [34]:
sentence_length.describe()

count    1.194000e+03
mean     3.643216e+13
std      4.814411e+13
min      2.739726e-03
25%      2.000000e+01
50%      5.000000e+01
75%      1.000000e+14
max      1.000000e+14
Name: Imposed Length, dtype: float64

In [35]:
with_sentence_lengths = revised_la_ho_data.assign(
    sentence_length=sentence_length
)

Now, we can ID the most serious offense for each person (allowing for ties) as follows:

In [36]:
def most_serious(df, doc_no):
    person_charges = df.loc[
        df["DOC Number"] == doc_no
    ]
    max_sentence = person_charges["sentence_length"].max()
    return person_charges.loc[
        person_charges["sentence_length"] == max_sentence
    ]

In [37]:
def is_most_serious(df, row):
    most_serious_off = most_serious(df, row["DOC Number"])
    return row["Offense Statute"] in most_serious_off["Offense Statute"].values

In [38]:
is_most_serious = with_sentence_lengths.apply(
    lambda row: is_most_serious(with_sentence_lengths, row), axis=1
)

In [39]:
is_most_serious.sum()

1056

Now we can filter the data to only include the most serious offenses for each person:

In [40]:
only_most_serious = with_sentence_lengths.assign(
    most_serious = is_most_serious
).loc[is_most_serious]

And then re-run the calculations from above:

In [41]:
by_doc_no_most_serious = only_most_serious.groupby("DOC Number")

In [42]:
len(by_doc_no_most_serious.groups)

884

#### First Degree Murder

In [43]:
by_doc_no_most_serious["first_degree_murder"].any().sum()

21

#### Second Degree Murder

In [44]:
by_doc_no_most_serious["second_degree_murder"].any().sum()

40

#### Rape

In [45]:
by_doc_no_most_serious["rape"].any().sum()

51

#### Drugs

In [46]:
by_doc_no_most_serious["drugs"].any().sum()

111

#### Posssession

In [47]:
by_doc_no_most_serious["poss"].any().sum()

49

#### PWID

In [48]:
by_doc_no_most_serious["pwid"].any().sum()

34

#### Distribution

In [49]:
by_doc_no_most_serious["dist"].any().sum()

31

### Most Common Top Charges (not filtered by unique)

In [50]:
only_most_serious["Offense Statute"].value_counts().head(25)

14:64        195
14:95.1       75
40:967        72
14:31         56
14:64.1       53
14:30.1       40
14:60         38
40:966        32
14:42.1       31
14:62         30
14:34         30
14:30         22
14:65         20
14:44.1       20
14:34.1       19
14:42         17
14:81.2       15
14:65.1       15
14:78.1       13
14:64.3       13
14:62.2       13
14:108.1C     12
14:402        11
14:34.7       11
14:43.1       11
Name: Offense Statute, dtype: int64

In [48]:
with_sentence_lengths.loc[
    with_sentence_lengths["first_degree_murder"]
    & ~is_most_serious
]

,Last Name,First Name,Middle Initial,DOC Number,Gender,Race,Current Age,Birth Date,Offense Statute,Modifier 1,...,Imposed Length,Overall Sentence Length,Parish Literal,first_degree_murder,rape,drugs,poss,pwid,dist,sentence_length
362,PITTMAN,BILLY,M,110215,M,W,57.20,19640919,14:30,H,...,1000000,LIFE,W. FELICIANA,True,False,False,False,False,False,100.0
369,EDGAR,JAMES,NaN,110300,M,B,59.15,19621008,14:30,H,...,0500000,LIFE,ORLEANS,True,False,False,False,False,False,50.0
